In [1]:
import random


In [13]:
#input: path - list of visited nodes    mat - adjancency matrix of the graph( mat[i][j] contains the cost of the edge from i to j)
#output: the total cost of path
# Theta(len(path))
def fitness(path,mat):
    sum=0
    for i in range(0,len(path)): # finding the cost of each edge and adding it to the total sum
        sum+=int(mat[path[i-1]-1][path[i]-1])
    return sum

In [14]:
#input: start - the starting node( whatever we want from 1 to nr)     nr - nr of nodes in chromosome representation
#output: a randomized representation of a chromosome starting from the specified node 
def generateChromosome(start,nr):
    l = random.sample(range(1,nr+1),nr) # creating the random sample
    ind = l.index(start) # swapping the 'start' node to the first position
    aux=l[0]
    l[0]=l[ind]
    l[ind]=aux
    return l

In [20]:

class Chromosome:
    def __init__(self, mat,start):
        self.__mat = mat  # adjacency matrix
        self.__repres = [] # path from start back to it passing through each node once
        self.__start=start # start node
        self.__fitness = self.initialize() # the fitness of this chromosome
     
    @property
    def repres(self):
        return self.__repres
    
    @property
    def fitness(self):
        return self.__fitness 
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l 
    
    @fitness.setter 
    def fitness(self, fit = 0.0):
        self.__fitness = fit 
    
    def initialize(self):
        # we generate a random chromosome
        self.__repres = generateChromosome(self.__start,len(self.__mat))
        return fitness(self.__repres,self.__mat)
    
    def crossover(self, c): # we use crossover to combine 2 chromosomes using '2 breaking point' method
        #picking 2 random indexes in out represenation
        pos1 = random.randint(-1, len(self.__mat) - 1)
        pos2 = random.randint(-1, len(self.__mat) - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1 
        k = 0
        
        newrepres = self.__repres[pos1 : pos2] # representation from index1 to index2 remains the same
        
        for el in c.__repres[pos2:] +c.__repres[:pos2]: #inserting the remaining elements
            if (el not in newrepres):
                if (len(newrepres) < len(self.__mat) - pos1):
                    newrepres.append(el)
                else:
                    newrepres.insert(k, el)
                    k += 1

        offspring = Chromosome(self.__mat,self.__start) # generate new chromosome
        
        ind = newrepres.index(self.__start)
        aux=newrepres[0]
        newrepres[0]=newrepres[ind]
        newrepres[ind]=aux
        
        offspring.repres = newrepres # add the resulted representation to the chromosome to be returned
        return offspring
    
    def mutation(self): # mutation function is used for escaping local minimums... it just does some slightly changes to the chromosome
        pos1 = random.randint(1, len(self.__mat) - 1)
        pos2 = random.randint(1, len(self.__mat) - 1)
        #pos3 = random.randint(1, len(self.__mat) - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1
        el = self.__repres[pos2]
        del self.__repres[pos2]
        self.__repres.insert(pos1 + 1, el)
    
    def __str__(self):
        return '\nChromo: ' + str(self.__repres) + ' has fit: ' + str(self.__fitness)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres and self.__fitness == c.__fitness

In [21]:
#GA is the 'main' class. We use it`s functions a specified number of times to drive the genethic algorithm towards the result

class GA:
    def __init__(self,param):
        self.__param = param # parameters of the GA. 
        self.__population = [] # list of param['popSize'] chromosomes
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self): # initializes the population
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__param['mat'],self.__param['start'])
            self.__population.append(c)
    
    def evaluation(self): # calculates the fitness of every chromosome in population
        for c in self.__population:
            c.fitness = fitness(c.repres,self.__param['mat'])
            
    def bestChromosome(self): # returns the chromosome with best fitness
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self): # returns the chromosome with worst fitness
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self): # returns a random index in one specific chromosome
        pos1 = random.randint(0, len(self.population) - 1)
        pos2 = random.randint(0, len(self.population) - 1)
        
        try:
            if (self.__population[pos1].fitness < self.__population[pos2].fitness): # we pick the best chromosome out of the randomly selected two
                return pos1
            else:
                return pos2 
        except:
            return pos1
    
    # oneGeneration and oneGenerationElitism makes one step towards the result.
    # oneGeneration: picks two random chromosomes in the population for popSize times and applyes crossover on them,
    #                mutates the result and adds the resulted chromosome to a new population
    # oneGenerationElisitm: does the same thing, except it preserves the best chromosome out of every population
    # although those 2 functions seem not very different , they are used for different problems and purposes. In or case we use Elitism.
    
    def oneGeneration(self): 
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            
            p2 = self.__population[self.selection()]
            
            off = p1.crossover(p2)
            
            if off.fitness>p1.fitness or off.fitness>p2.fitness: 
                off.mutation()
            
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        

In [22]:
#input: filename - string representing the file to read the data from
#output: the parameters that GA needs
def data(filename):
    f=open(filename,"r")
    a=f.read()
    a=a.split()

    mat=[]
    for i in range(1,int(a[0])+1): 
        mat.append(a[i].split(",")) 
    param={}
    param["mat"]=mat
    param["popSize"]=len(mat)*2
    param["noNodes"]=len(mat)
    param["start"]=1
    return param

In [23]:
# main function
def run(numberOfSteps,filename):
    param = data(filename)
    #print(param)
    ga = GA(param)
    ga.initialisation();
    ga.evaluation();
    best = ga.bestChromosome()
    last =[] #if we want to visualize how the GA converges to the result
    
    for _ in range(numberOfSteps): # we create new populations 
        ga.oneGenerationElitism()
        last.append(ga.bestChromosome())     
        ga.worstChromosome().mutation() # mutating the worst chromosome with the hope that it becomes better and helps future populations
        
        if ga.bestChromosome().fitness < best.fitness:
            best=ga.bestChromosome()

    return best.fitness # overall result
    

In [24]:
# just to measure how many times our algorithm returns the best possible path... assuming we know the answer.
# For hard.txt I know it`s 41
def accuracy():
    c=0
    for _ in range(100):
       
        if run(200,"hard.txt") ==41:
            c+=1
    print(str(c)+"%")
accuracy()

28%


In [29]:
# Keep in mind that this algorithm is based on random actions therefore it is not certain that it gives the best path all the times.
# Sometimes it just gets very close but doesn`t find the best path. Although, since we are using AI, we expect to not have the best answer everytime.
# Use example:
print(run(200,'hard.txt'))

41
